In [1]:
%matplotlib widget
#coding=utf8

import pandas as pd
import math
import matplotlib.pyplot as plt
import geopandas
import numpy as np
import requests
import json
import os
import geoplot as gplt
import geoplot.crs as gcrs
#from geoplot import HeatMap
import pandas_profiling
import folium

from folium.plugins import HeatMap
from chardet import detect
from scipy.interpolate import SmoothBivariateSpline
from shapely import geometry

'''define the Geoinfo function'''
def coords(city):
    url = 'https://restapi.amap.com/v3/geocode/geo'
    
    params = { 'key': 'd56c2541d4a349b6fb38c6e2c02fc12a',
             'address': city}
    res = requests.get(url, params)
    jd = json.loads(res.text)
    
    return jd['geocodes'][0]['location'].split(',')

In [2]:
'''Change encode of csv'''
with open('location_dpt.csv', 'rb+') as fp:
    content = fp.read()
    encoding = detect(content)['encoding']
    content = content.decode(encoding).encode('utf-8')
    fp.seek(0)
    fp.write(content)

In [3]:
'''Read the model results'''
plt_data = pd.read_csv('tp_pd_S_pltdata.csv')
plt_data.profile_report(title='Plot Data')

Summarize dataset:   0%|          | 0/16 [00:00<?, ?it/s]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
'''Generate the Geo info'''
geo_origin = pd.read_csv('location_dpt.csv')
geo_origin.loc[:,'lat'] = 0.0
geo_origin.loc[:,'long'] = 0.0
for i in range(geo_origin.shape[0]):
    locations = coords(geo_origin.loc[i, 'location'])
    geo_origin.loc[i,'long'] = locations[0]
    geo_origin.loc[i,'lat'] = locations[1]

In [5]:
'''Merge the data'''
plt_geo_data = plt_data.merge(geo_origin, how='inner', left_on='levels', right_on='dpt')
plt_geo_data = plt_geo_data.loc[:,['values', 'w', 'long', 'lat']].copy()
plt_geo_data = plt_geo_data.astype(float)
plt_geo_data['w'] = plt_geo_data['w']/plt_geo_data['w'].sum()

In [6]:
plt_model = SmoothBivariateSpline(plt_geo_data['long'],
                                  plt_geo_data['lat'],
                                  plt_geo_data['values'])

In [7]:
china = geopandas.read_file('China.json')
china['geometry'] = china['geometry'].buffer(0)
df = geopandas.GeoDataFrame(plt_geo_data[['long', 'lat', 'values']])
xy = [geometry.Point(xy) for xy in zip(df['long'], df['lat'])]
df['geometry'] = geopandas.GeoDataFrame(xy)

/Users/chenxuyi/opt/anaconda3/envs/h2oai/lib/python3.6/site-packages/geopandas/geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [8]:
ax = gplt.webmap(china, projection=gcrs.WebMercator(), figsize=(20,20), zoom=5)
gplt.pointplot(df, hue='values', projection=gcrs.WebMercator(), cmap='inferno_r',legend=True,ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<GeoAxesSubplot:>

In [9]:
m = folium.Map(location=[35.3, 100.6], zoom_start=4, zoom_min=4)
HeatMap(plt_geo_data[['lat', 'long', 'values']].values).add_to(m)
m

In [10]:
location = coords(input())
Pred_values = np.round(plt_model.ev(location[0], location[1]),3)
print(location)
print(Pred_values)

 镇江


['119.425836', '32.187849']
1.021


In [11]:
n_step = 100
i = 0 
lat_step = (plt_geo_data['lat'].max()-plt_geo_data['lat'].min())/n_step
long_step = (plt_geo_data['long'].max()-plt_geo_data['long'].min())/n_step
lat_start = plt_geo_data['lat'].min() + lat_step
long_start = plt_geo_data['long'].min() + long_step
grid_data = pd.DataFrame(columns=['lat', 'long', 'values'])
wchina = china['geometry'][0]
for s in range(1, len(china)):
    wchina = wchina.union(china['geometry'][s])
for j in range(n_step):
    for k in range (n_step):
        long = long_start + j * long_step
        lat = lat_start + k  * lat_step
        if geometry.Point(long, lat).within(wchina):
            grid_data.loc[i, 'lat'] = lat
            grid_data.loc[i, 'long'] = long
            grid_data.loc[i, 'values'] = plt_model.ev(long, lat).item()
            i += 1 
    

In [12]:
m_grid = folium.Map(location=[35.3, 100.6], zoom_start=4)
HeatMap(grid_data.values).add_to(m_grid)
m_grid

In [13]:
grid_data

,lat,long,values
0,21.7771,100.375,1.01765
1,22.0303,100.375,1.01101
2,22.2835,100.375,1.00508
3,22.5366,100.375,0.999847
4,22.7898,100.375,0.995273
...,...,...,...
6541,45.5766,129.633,0.985691
6542,45.8298,129.633,0.986795
6543,46.0829,129.633,0.988296
6544,46.3361,129.633,0.990205
